In [ ]:
import csv
import cv2
import numpy as np
import os
import glob
import shutil
from sklearn.utils import shuffle
from collections import OrderedDict
from shutil import copyfile

from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.optimizers import SGD
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [ ]:
#ensure GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
#check dim ordering
print(k.image_data_format())#integer seed for any randomnessnp.int64(np.floor(time.time()))

In [ ]:
# Initialize session
import tensorflow as tf
sess = tf.InteractiveSession()

# Some tensor we want to print the value of
a = tf.constant([1.0, 3.0])

# Add print operation
a = tf.Print(a, [a], message="This is a: ")

# Add more elements of the graph using a
b = tf.add(a, a).eval()
Now, whenever we evaluate the whole graph, e.g. using b.eval(), we get:

I tensorflow/core/kernels/logging_ops.cc:79] This is a: [1 3]

In [ ]:
bosch_label_dict = {
    'red': [],
    'yellow': [],
    'green': [],
    'nolight': []
}

label_set = set(['red', 'green', 'yellow'])
for image in data_yaml:
    if len(image['boxes']) > 5:
        continue
    elif not image['boxes']:
        bosch_label_dict['nolight'].append(image['path'])
    else:
        image_labels = Counter([x['label'].lower() for x in image['boxes'] if x['label'].lower() in bosch_label_dict])
        if len(image_labels.most_common()) > 0:
            chosen_label = image_labels.most_common()[0][0]
            bosch_label_dict[chosen_label].append(image['path'])
    

In [ ]:
working_dir = './data/bosch_traffic/full_set/'
if not os.path.exists(working_dir): os.mkdir(working_dir)
for label, files in bosch_label_dict.items():
    label_dir = working_dir + label
    if not os.path.exists(label_dir): os.mkdir(label_dir)
    for file in files:
        shutil.copy2('./data/bosch_traffic/' + file, label_dir + '/' + os.path.basename(os.path.normpath(file)))

In [ ]:
def build_data_dir_for_keras(seed, data_base_uri):
    data_dir = os.getcwd() + data_base_uri
    full_data_dir = data_dir + 'full_set/'
    working_data_dir = data_dir + 'working_set/'

    if not os.path.exists(working_data_dir): os.mkdir(working_data_dir)
        
    for subset_label in ['train/', 'val/', 'test/']:
        subset_data_uri = working_data_dir + subset_label
        if not os.path.exists(subset_data_uri): os.mkdir(subset_data_uri)

    for class_name in os.listdir(full_data_dir):
        for subset_label in ['train/', 'val/', 'test/']:
            subset_label_data_uri = working_data_dir + subset_label + class_name + '/'
            if not os.path.exists(subset_label_data_uri): os.mkdir(subset_label_data_uri)
        D_class = []
        
        for filename in glob.glob(full_data_dir + class_name + '/*.png'):
            D_class.append(filename)

        D_class = shuffle(D_class, random_state=seed)
    
        class_count = len(D_class)
        train_val_index = int(np.floor(len(D_class) * 0.70))
        val_test_index = int(np.floor(len(D_class) * 0.85))

        # can be abbreviated considerably
        for file in D_class[:train_val_index]:
            shutil.copy2(file, 
                         working_data_dir + '/train/' + class_name + '/' + os.path.basename(os.path.normpath(file)))
        for file in D_class[train_val_index:val_test_index]:
            shutil.copy2(file, 
                         working_data_dir + '/val/' + class_name + '/' + os.path.basename(os.path.normpath(file)))
        for file in D_class[val_test_index:]:
            shutil.copy2(file, 
                         working_data_dir + '/test/' + class_name + '/' + os.path.basename(os.path.normpath(file)))

In [ ]:
data_dir = os.getcwd() + '/data/final_sim_data'
full_data_dir = data_dir + '/full_set'
working_data_dir = data_dir + '/working_set'



for class_name in os.listdir(full_data_dir):
    D_class = []
    
    for filename in glob.glob(full_data_dir + '/' + class_name + '/*.png'):
        D_class += filename
    
    D_class = shuffle(np.array(D_class))
    
    class_count = len(D_class)
    train_val_index = int(np.floor(len(D_class) * 0.70))
    val_test_index = int(np.floor(len(D_class) * 0.85))
Contents
    for file in D_class[:train_val_index]:
        shutil.copy2(filename, 
                     working_data_dir + '/train/' + class_name + os.path.basename(os.path.normpath(filename)))
    for file in D_class[train_val_index:val_test_index]:
        shutil.copy2(filename, 
                     working_data_dir + '/val/' + class_name + os.path.basename(os.path.normpath(filename)))
    for file in D_class[val_test_index:]:
        shutil.copy2(filename, 
                     working_data_dir + '/test/' + class_name + os.path.basename(os.path.normpath(filename)))

# VISUALIZTION TECHNIQUES

## T-SNE conversion to lower dimensions with grid arranged viewing

## Maximal activation of kernels (hottest images)

## Visualize filters, look for smoothness (maybe even autocalculate it) -> training duration and overfitting

## activation maps of images -> find dead filters, also automatically

## occulsion probability heatmaps -> see what's really learning the class in the image

## TENSOR BOARD SHIT

In [ ]:
# helper stolen 
def get_nb_files(directory):
  """Get number of files by searching directory recursively"""
  if not os.path.exists(directory):
    return 0
  cnt = 0
  for r, dirs, files in os.walk(directory):
    for dr in dirs:
      cnt += len(glob.glob(os.path.join(r, dr + "/*")))
  return cnt

In [ ]:
# a bunch of junk code for pushing around earlier data that I'm not sure I won't need again...

# find . -type f -name '*.png' -delete

# !! ls data/TrafficLightDataset/working_data/

# with open('./data/TrafficLightDataset/labels_num.csv') as labels:
#     c_labels = OrderedDict(csv.reader(labels))

# X_labels = np.array(list(c_labels.keys()))
# Y = np.array(list(c_labels.values()))
# shuffle(X_labels, Y)

# images = []
# for label in X_labels:
#     loc = "/home/dieslow/WORKSPACE/PROJECTS/CARZ/nanodegree/Capstone/ros/src/tl_detector/data/TrafficLightDataset/image_data/" + label + ".png"
#     image = cv2.imread(loc, 1)
# #     image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) 
#     # May not need to do resizing... BUT SHOULD VIEW IMAGES
# #     resized_image = cv2.resize(image, (256, 256)) 
#     images.append(image)
# X = np.array(images)
# print(X.shape)

# train_val_index = int(np.floor(len(Y) * 0.85))

# D = list(zip(X, Y))
# D_train = D[:train_val_index]
# D_val = D[train_val_index:]
# X_train, Y_train = zip(*D_train)
# print(len(X_train))
# X_val, Y_val = zip(*D_val)
# print(len(X_val))

# images = []
# for x, y in D_train:
#     loc = "/home/dieslow/WORKSPACE/PROJECTS/CARZ/nanodegree/" + \
#           "Capstone/ros/src/tl_detector/data/TrafficLightDataset/working_data/train/" + \
#           str(y) + "/" + str(np.random.randint(100000000, 999999999)) + ".png"
#     cv2.imwrite(loc, x)
    
# for x, y in D_val:
#     loc = "/home/dieslow/WORKSPACE/PROJECTS/CARZ/nanodegree/" + \
#           "Capstone/ros/src/tl_detector/data/TrafficLightDataset/working_data/val/" + \
#           str(y) + "/" + str(np.random.randint(100000000, 999999999)) + ".png"
#     cv2.imwrite(loc, x)

# BASIC TEMPLATE  FOR TRANSFER LEARNING

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 299, 299

# TRAIN_DATA_DIR = "data/TrafficLightDataset/working_data/train"
# VAL_DATA_DIR = "data/TrafficLightDataset/working_data/val"
TRAIN_DATA_DIR = "./data/TrafficLightDataset/big_working_data/train"
VAL_DATA_DIR = "./data/TrafficLightDataset/big_working_data/val"
TEST_DATA_DIR = "./data/TrafficLightDataset/big_working_data/test"

NB_TRAIN_SAMPLES = get_nb_files(TRAIN_DATA_DIR)
NB_VAL_SAMPLES = get_nb_files(VAL_DATA_DIR)
NB_TEST_SAMPLES = get_nb_files(TEST_DATA_DIR)

BATCH_SIZE = 40
EPOCHS_TRANSFER = 5
EPOCHS_FINE_TUNE = 5

TRAIN_STEPS_PER_EPOCH = int(np.ceil(NB_TRAIN_SAMPLES / BATCH_SIZE))
VAL_STEPS_PER_EPOCH = int(np.ceil(NB_VAL_SAMPLES / BATCH_SIZE))
TEST_STEPS_PER_EPOCH = int(np.ceil(NB_TEST_SAMPLES / BATCH_SIZE))

FROZEN_LAYERS = 172



# USE TF COLLECTIONS FOR KEY VALUE GROUPED METADATA LIKE LOSS COMPONENTS

# IN TF AN EDGE IS A TENSOR AND A NODE IS A OPERATION!!!!!

## FOR LOADING AND SAVING

In [ ]:

saver = tf.train.Saver()    
saver.save(sess, './models/workingish_beta_1.0')
# OPTIONAL: Apply the trained model to a video

tf.reset_default_graph()
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('my_test_model-1000.meta')
#     loader = tf.train.Saver()
    saver.restore(sess, './models/checkpoint')
    
    graph = tf.get_default_graph()
    w1 = graph.get_tensor_by_name("w1:0")
    w2 = graph.get_tensor_by_name("w2:0")
    feed_dict ={w1:13.0,w2:17.0}

    #Now, access the op that you want to run. 
    op_to_restore = graph.get_tensor_by_name("op_to_restore:0")

print sess.run(op_to_restore,feed_dict)
#This will print 60 which is calculated 

In [ ]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
#     rescale = 1./255,
#     horizontal_flip = True,
#     zoom_range = 0.4,
#     width_shift_range = 0.3,
#     height_shift_range=0.3,
#     rotation_range=20
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
#     horizontal_flip = True,
#     rescale = 1./255,
#     fill_mode = "nearest"
#     zoom_range = 0.3,
#     width_shift_range = 0.3,
#     height_shift_range=0.3,
#     rotation_range=30
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = BATCH_SIZE)

validation_generator = test_datagen.flow_from_directory(
    VAL_DATA_DIR,
    batch_size = BATCH_SIZE,
    target_size = (IMG_HEIGHT, IMG_WIDTH))

test_generator = test_datagen.flow_from_directory(
    VAL_DATA_DIR,
    batch_size = BATCH_SIZE,
    target_size = (IMG_HEIGHT, IMG_WIDTH))


In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape = (IMG_HEIGHT, IMG_WIDTH, 3))

# + global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# + FC layer
x = Dense(1024, activation='relu')(x)

# + log laye
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (most places, including the inceptv3 paper recommended RMSProp, 
#  but I had a lot of premature convergence issues with it. to be fair, I switched to
#  adam experimenting around before I spent anytime optimizing LR, so who knows. converges
#  quick enough for Anand's data as it is)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("inceptionv3.h5", monitor='val_loss', verbose=1, save_best_only=True, 
                             save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

# training for transfer learning
model.fit_generator(
    train_generator,
    steps_per_epoch = TRAIN_STEPS_PER_EPOCH,
    epochs = EPOCHS_TRANSFER,
    validation_data = validation_generator,
    validation_steps = VAL_STEPS_PER_EPOCH,
    callbacks = [early, checkpoint],
    verbose=True, 
    class_weight='auto',
    use_multiprocessing=True)

# start fine-tuning conv layers in addition to FC. 
# freeze the bottom FROZEN_LAYERS layers, train the rest
for layer in model.layers[:FROZEN_LAYERS]:
   layer.trainable = False
for layer in model.layers[FROZEN_LAYERS:]:
   layer.trainable = True

# compile
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# training for fine-tuning conv layers
model.fit_generator(
    train_generator,
    steps_per_epoch = TRAIN_STEPS_PER_EPOCH,
    epochs = EPOCHS_FINE_TUNE,
    validation_data = validation_generator,
    validation_steps = VAL_STEPS_PER_EPOCH,
    callbacks = [early, checkpoint], #
    verbose=True,
    class_weight='auto',
    use_multiprocessing=True)


print(model.metrics_names)
model.evaluate_generator(
    test_generator,
    steps = TEST_STEPS_PER_EPOCH,
    use_multiprocessing=True)